In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import gamma
import seaborn as sns

def levy_flight_steps(n_steps, alpha=1.5, beta=0, mu=0, sigma=1):
    """
    Generate Lévy flight step sizes using the Chambers-Mallows-Stuck method.

    Parameters:
    -----------
    n_steps : int
        Number of steps to generate
    alpha : float
        Stability parameter (0 < alpha <= 2). Lower values = heavier tails
        alpha = 1.5 is typical for Lévy flights
    beta : float
        Skewness parameter (-1 <= beta <= 1)
    mu : float
        Location parameter
    sigma : float
        Scale parameter (sigma > 0)

    Returns:
    --------
    numpy.ndarray
        Array of Lévy distributed step sizes
    """
    # Generate uniform random variables
    U = np.random.uniform(-np.pi/2, np.pi/2, n_steps)
    W = np.random.exponential(1, n_steps)

    if alpha == 1:
        # Special case for alpha = 1 (Cauchy distribution when beta = 0)
        steps = (2/np.pi) * ((np.pi/2 + beta*U) * np.tan(U) -
                            beta * np.log((np.pi/2)*W*np.cos(U)/(np.pi/2 + beta*U)))
    else:
        # General case using Chambers-Mallows-Stuck method
        B = np.arctan(beta * np.tan(np.pi * alpha / 2)) / alpha
        S = (1 + beta**2 * np.tan(np.pi * alpha / 2)**2)**(1/(2*alpha))
        steps = S * (np.sin(alpha * (U + B)) / (np.cos(U)**(1/alpha))) * \
                (np.cos(U - alpha * (U + B)) / W)**((1-alpha)/alpha)

    return mu + sigma * steps

def levy_walk_2d(n_steps, alpha=1.5, dt=1.0):
    """
    Simulate a 2D Lévy walk (random walk with Lévy-distributed step sizes).

    Parameters:
    -----------
    n_steps : int
        Number of steps in the walk
    alpha : float
        Stability parameter for Lévy distribution
    dt : float
        Time step

    Returns:
    --------
    tuple
        (x_positions, y_positions) arrays of the walk trajectory
    """
    # Generate step sizes from Lévy distribution
    step_sizes = np.abs(levy_flight_steps(n_steps, alpha=alpha))

    # Generate random directions (angles)
    angles = np.random.uniform(0, 2*np.pi, n_steps)

    # Calculate step components
    dx = step_sizes * np.cos(angles)
    dy = step_sizes * np.sin(angles)

    # Calculate cumulative positions
    x = np.cumsum(np.concatenate([[0], dx]))
    y = np.cumsum(np.concatenate([[0], dy]))

    return x, y

def analyze_levy_distribution(steps, alpha_theoretical):
    """
    Analyze and visualize the properties of generated Lévy steps.
    """
    plt.figure(figsize=(15, 10))

    # Plot 1: Histogram of step sizes
    plt.subplot(2, 3, 1)
    plt.hist(steps, bins=100, density=True, alpha=0.7, edgecolor='black')
    plt.xlabel('Step Size')
    plt.ylabel('Probability Density')
    plt.title(f'Distribution of Lévy Steps (α={alpha_theoretical})')
    plt.yscale('log')
    plt.xscale('log')

    # Plot 2: Log-log plot to show power law tail
    plt.subplot(2, 3, 2)
    hist, bin_edges = np.histogram(np.abs(steps), bins=50, density=True)
    bin_centers = (bin_edges[1:] + bin_edges[:-1]) / 2

    # Remove zeros for log plot
    mask = (hist > 0) & (bin_centers > 0)
    plt.loglog(bin_centers[mask], hist[mask], 'bo-', markersize=4)
    plt.xlabel('|Step Size|')
    plt.ylabel('Probability Density')
    plt.title('Power Law Tail (Log-Log Scale)')
    plt.grid(True, alpha=0.3)

    # Plot 3: Cumulative distribution
    plt.subplot(2, 3, 3)
    sorted_steps = np.sort(np.abs(steps))
    p = np.arange(1, len(sorted_steps) + 1) / len(sorted_steps)
    plt.loglog(sorted_steps, 1 - p, 'r-', linewidth=2)
    plt.xlabel('|Step Size|')
    plt.ylabel('P(X > x)')
    plt.title('Complementary CDF')
    plt.grid(True, alpha=0.3)

    # Plot 4: Sample Lévy walk trajectory
    plt.subplot(2, 3, 4)
    x, y = levy_walk_2d(1000, alpha=alpha_theoretical)
    plt.plot(x, y, 'b-', alpha=0.7, linewidth=0.8)
    plt.scatter(x[0], y[0], color='green', s=100, label='Start', zorder=5)
    plt.scatter(x[-1], y[-1], color='red', s=100, label='End', zorder=5)
    plt.xlabel('X Position')
    plt.ylabel('Y Position')
    plt.title('2D Lévy Walk Sample Path')
    plt.legend()
    plt.axis('equal')

    # Plot 5: Step size vs step number
    plt.subplot(2, 3, 5)
    plt.plot(range(len(steps)), np.abs(steps), 'k-', alpha=0.6)
    plt.xlabel('Step Number')
    plt.ylabel('|Step Size|')
    plt.title('Step Sizes Over Time')
    plt.yscale('log')

    # Plot 6: Statistics summary
    plt.subplot(2, 3, 6)
    plt.axis('off')

    # Calculate statistics
    mean_step = np.mean(np.abs(steps))
    median_step = np.median(np.abs(steps))
    std_step = np.std(steps)
    max_step = np.max(np.abs(steps))

    stats_text = f"""
    Lévy Flight Statistics (α = {alpha_theoretical})

    Sample Size: {len(steps):,}
    Mean |Step|: {mean_step:.4f}
    Median |Step|: {median_step:.4f}
    Std Dev: {std_step:.4f}
    Max |Step|: {max_step:.4f}

    Theoretical Properties:
    • Heavy tails for α < 2
    • Infinite variance for α < 2
    • Scale-free jumps
    • Superdiffusive behavior
    """

    plt.text(0.1, 0.9, stats_text, transform=plt.gca().transAxes,
             verticalalignment='top', fontfamily='monospace', fontsize=10)

    plt.tight_layout()
    plt.show()

# Main simulation and analysis
if __name__ == "__main__":
    # Set random seed for reproducibility
    np.random.seed(42)

    # Parameters
    n_samples = 10000
    alpha_values = [1.2, 1.5, 1.8]  # Different stability parameters

    print("Generating Lévy flight samples with different α values...\n")

    for alpha in alpha_values:
        print(f"=== Analysis for α = {alpha} ===")

        # Generate Lévy distributed steps
        levy_steps = levy_flight_steps(n_samples, alpha=alpha)

        # Analyze and visualize
        analyze_levy_distribution(levy_steps, alpha)

        # Print some basic statistics
        abs_steps = np.abs(levy_steps)
        print(f"Mean absolute step size: {np.mean(abs_steps):.4f}")
        print(f"95th percentile: {np.percentile(abs_steps, 95):.4f}")
        print(f"99th percentile: {np.percentile(abs_steps, 99):.4f}")
        print(f"Maximum step: {np.max(abs_steps):.4f}")
        print("-" * 50)

    # Demonstrate multiple 2D Lévy walks
    plt.figure(figsize=(12, 8))
    colors = ['blue', 'red', 'green', 'purple', 'orange']

    for i, alpha in enumerate([1.2, 1.5, 1.8]):
        plt.subplot(2, 2, i+1)

        # Generate multiple walks for comparison
        for j in range(5):
            x, y = levy_walk_2d(500, alpha=alpha)
            plt.plot(x, y, color=colors[j], alpha=0.7, linewidth=1)
            plt.scatter(x[0], y[0], color=colors[j], s=30, marker='o')
            plt.scatter(x[-1], y[-1], color=colors[j], s=30, marker='s')

        plt.title(f'Lévy Walks (α = {alpha})')
        plt.xlabel('X Position')
        plt.ylabel('Y Position')
        plt.axis('equal')
        plt.grid(True, alpha=0.3)

    # Comparison of different alpha values in one plot
    plt.subplot(2, 2, 4)
    for i, alpha in enumerate([1.2, 1.5, 1.8]):
        x, y = levy_walk_2d(1000, alpha=alpha)
        plt.plot(x, y, label=f'α = {alpha}', linewidth=1.5, alpha=0.8)
        plt.scatter(x[0], y[0], s=50, marker='o')
        plt.scatter(x[-1], y[-1], s=50, marker='s')

    plt.title('Comparison of Lévy Walks')
    plt.xlabel('X Position')
    plt.ylabel('Y Position')
    plt.legend()
    plt.axis('equal')
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    print("\nSimulation complete!")
    print("Lower α values produce more extreme jumps and heavier tails.")
    print("This demonstrates the characteristic long-tail behavior of Lévy flights.")